# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

參考

enumerate

https://www.runoob.com/python/python-func-enumerate.html

beautifulsoup

https://kknews.cc/zh-tw/news/lr2zvn9.html

Selenium

https://365airsoft.com/zh-CN/questions/1434228/python-selenium-webdriver?utm_source=landcareweb.com&utm_medium=Redirect

https://www.itread01.com/content/1550384115.html

http://yhhuang1966.blogspot.com/2018/05/python-selenium_27.html

https://ithelp.ithome.com.tw/articles/10222029

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome("../chromedriver")
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    
    '''
     # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        browser.execute_script('alert("ToBottom")') #在網頁中跳出警告標語
        print('到達頁面底端')
        break

    print('網頁更新中...')
    last_height = new_height

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [4]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        (4)  .text() 他會抓某一個 DOM 底下所有的文字
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [6]:
#html = browser.page_source
#soup = BeautifulSoup(html, "lxml")
#all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})
#
#news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})
#type(news_block[0])

In [7]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 2020年貨幣及財政政策都放鬆！中國社科院：GDP增速約6%

記者邱茀濱／綜合報導
中國社科院世界經濟與政治研究所國際投資研究室主任張明指出，2020年中國經濟仍將繼續尋底，GDP增速即使破6%，也會在6%附近；CPI增速則有望在二、三季度明顯回落，「滯脹」的短期格局將會有所修正。
張明在最新文章中表示，2020年對於中國是穩增長、調結構與控風險的微妙平衡之年，預計明年政府仍將實施逆週期宏觀經濟政策，但政策力度與政策重點均在審慎考慮的範圍內，而且可能隨著經濟增速、風險暴露的演進而調整。
「2020年，無論是完成全面構建小康社會的目標，還是完成10年內GDP總量翻番的目標，都問題不大。但還是不能忽視結構調整與防範系統性風險的中長期任務」，張明稱，「如何解決地方債問題、以及與地方債關聯密切的房地產問題，將在未來五年內繼續考驗中國政府的智慧、勇氣與決心。」
至於財政政策方面，張明預料明年寬鬆力度將會進一步加大，公共預算赤字率料將上升至約3%，專項債發行規模可能超過3萬億元人民幣，並會要求地方政府將大部分專項債用於基礎設施建設。
此外，還會調低部分基建項目的資本金比例，以吸引信貸資金與社會資金的參與；同時，加大對地方政府隱性債務的置換處理力度。「考慮到財政前置作用與通貨走勢，2020年可能呈現出財政上半年發力、貨幣下半年發力的特點。」張明稱。
針對貨幣政策，張明預計2020年仍將有2至3次的降準，最近的一次降準可能發生在春節之前或者兩會前後。「明年仍將延續溫和放鬆基調，降準、公開市場操作與降息三者仍將有機結合起來。」
►「IG討論爆款」～超殺新年禮物來了！
----------------------------------------------------------------------

[1] 2019年資金短缺、工資遲發！海航董事長：明年是化解流動性風險決勝之年

記者邱茀濱／綜合報導
「2020年是海航化解流動性風險的決勝之年。」海航集團30日發布了董事長陳峰2020年新年獻詞，他坦承2019年海航有資金短缺，工資遲發、緩發的情況，但每位海航人都敢於擔當，勇於創新，讓每一顆螺絲釘都安全牢固，他向大家致以最崇

▲國產署明年將釋出7宗國有地標租，提供業者設置太陽能光電發電設備使用（示意圖／記者林悅翻攝）
記者吳靜君／台北報導
為了推動太陽能光電政策，國產署今（30）日表示，為了持續配合國家政策以及太陽能光電業者的需求，明（2020）年預計釋出7宗國有地標租，國產署表示，該等國有土地區位及基地條件俱佳，預計將在農曆年後在2月、4月兩批釋出、標租，歡迎有興趣的光電業者踴躍投標。
國產署表示，今（2019）年標租國有土地供太陽能光電發電設置使用情形良好，今年共標租2批，第一批標脫台南善化及高雄美濃土地，分別年收入租金有53萬元和62萬元；第二批再度標脫2宗，分別為台中后里及嘉義布袋，年收租金分別為309萬元與157萬元；此4宗土地標租光電案，面積合計6.67公頃。
國產署表示，為了配合推動台陽能光電政策，依據國有非公用不動產出租管理辦法，國有地可以標租給民間設置太陽能光電設備使用，除了去年的4見順利標脫。國產署目前再選出的國有地包含新竹縣1處、苗栗縣1處、台中市1處、雲林縣1處、嘉義縣2處及高雄市1處，共計7宗要預計明年標租給民間業者運用。
7宗國有地釋出共計11.19公頃，其中台中有1萬2429平方公尺、新竹湖口鄉有1萬4357平方公尺、苗栗有2059平方公尺、雲林1萬477平方公尺、高雄則是1萬2800平方公尺、嘉義的兩處分別釋出3萬5450平方公尺以及2萬402平方公尺。
國產署表示，該7宗國有地預計分兩批標租，大約都是在農曆年後，分別2月與4月標租，也希望有興趣的光電業者評估並且投標。目前國產署提供國有地設置太陽能光電有4種方式，從2018年開辦至今，申請開發（由民間申請籌設、國產署發同意書同意委託經營）有15件；已經同意委託經營有1件；委託改良利用（國產署提供土地，交由能源局或地方政府招商）有2件；標租則有4件。
----------------------------------------------------------------------

[13] 三星S系列新機傳出三種螢幕尺寸！　螢幕更新率挑戰120Hz　且能自動切換

▲Samsung Galaxy S11。（圖／取自91 mobiles）
記者謝仁傑／綜合報導
根據外媒報導，以洩露新產品資訊而聞名的IT部落客Evan Blass發文指出，三星將於明年推出的旗艦機款Galaxy S11系列一共有三款

記者姚惠茹／台北報導
TrendForce光電研究(WitsView)今（30）日公布，根據「新型顯示技術成本分析報告」指出，隨著LED打件速率的提升和Mini LED晶片價格下滑，65吋4K電視若採用Mini LED背光，整個顯示器模組的生產成本預估較2018年下降約5~10%，對於Mini LED推廣將有所助益。
TrendForce研究副理李志豪表示，Mini LED自從2017年下半年成為市場焦點以來，從技術浮現到進入商業化經歷約兩年的發展，相比過去的顯示技術演進過程來說算是非常有效率。
李志豪指出，由於Mini LED依然是在LCD的架構之下，等於將可進行直下式區域調光的背光規格推升到極致，是現有技術的一次重要升級，隨著TCL今年第4季率先在北美推出65吋和75吋搭載Mini LED背光的電視後，消費者在高階電視上也多了一個新的選擇。
▲65吋4K電視不同技術成本比較。（圖／TrendForce提供）
65吋UHD 4K電視在不同技術的成本表現上，高階的側入式背光顯示器模組生產成本約在350美元，而採用被動式驅動的Mini LED背光（LED使用顆數約16,000顆）的顯示器模組則約在650~690美元之間；至於海信今年所發表的疊屏電視(Dual Cell)，推算其顯示器模組的成本約在630美元左右，所以雖然Mini LED電視的生產成本還是略高，但已開始具備市場競爭力。
TrendForce指出，今年量產的Mini LED背光電視產品多半採用被動式驅動(PM)，但因為LED驅動IC的用量會隨著背光分區數的提升而同步增加，除了需要用比較大的驅動板，生產成本也會提高。
TrendForce表示，反之採用主動式驅動(AM) Mini LED背光的顯示器，在分區數越多的情況下越能顯現其優勢，除了成本更有競爭力之外，規格也能做到媲美OLED的水準，因此像是群創、友達、京東方及華星光電等面板廠積極開發的主動式驅動產品，有望推升Mini LED背光在高階電視的市佔率。
----------------------------------------------------------------------

[27] 台積電跌逾1%　台股翻黑下跌38點失守12,100點

▲ 台股。（資料照／記者湯興漢攝）
記者李瑞瑾／綜合報導
台股今（30）日早盤在平盤上震盪，但

▲ YAHOO公布2019年第四季與全年熱門台股榜。（圖／YAHOO提供）
記者李瑞瑾／綜合報導
歲末年終倒計時！回顧2019年，Yahoo奇摩股市App今（30）日公佈第四季「十大爆紅台股榜」、「五大爆紅ETF榜」，以及2019年度「十大熱門台股榜」。看到2019年度「十大熱門台股榜」，再度由電子股霸榜，且鴻海、台積電連莊冠、亞軍。
2019年十大熱門台股榜
在科技產品中不可或缺的印刷電路板（PCB），由欣興（3037）、華通（2313）分別進榜第6、7名。而記憶體相關股號的熱度持續延燒至今年，包括名列第3名的旺宏（2337）及第9名的華邦電（2344），均預期2020年記憶體市況可望大致穩定，並帶動相關產品有機會漲價。至於面板大廠群創（3481），因為明年面板報價可望回春，重新受到投資人青睞。
▲ 台積電股價在今年頻頻刷新歷史新高。（圖／記者屠惠剛攝）
2019年第四季十大爆紅台股榜
至於今年第四季爆紅台股榜，包括順德（2351）、瓦城（2729）、大車隊（2640）、根基（2546）、雲品（2748）和互盛電（2433）。此六檔個股有兩大共通點，首先是擁有強勢品牌，各自有深厚的經營基礎，像是順德為「SDI手牌文具」的製造商等；其次是近年均有穩定、甚至逐年成長的現金股息收益，其中，順德股息從2元逐年成長至2.8元。
其他4席則是台開（2841）、立萬利（3054）、福懋油（1225）和佳能（2374）等漲幅相對落後的個股，展現了部分更積極的投資人，持續關注個股動態、等待翻轉獲利時機。
2019年第四季五大爆紅ETF榜
今年第四季爆紅ETF類型十分多元，奪下本季爆紅ETF首位的富邦美國特別股（00717），標榜特色是在美股頻頻創新高之際，股息殖利率仍有5.29%，加上在償債順序上特別股優先於普通股，令投資人在較低的總費用率下，不光是得到股息收益，還能佈局性質接近債券的資產。元大道瓊白銀（00738U）今年曾經出現一波強勁的漲幅，令投資人關注世界第二貴金屬重啟多頭的可能性，排名第2名。
▲ 美股頻創新高，吸引投資人關注。（圖／路透社）
儘管台股今年以來的漲幅在亞洲數一數二，因應MSCI調整指數成份股引起投資人關注，讓以往熱門的元大台灣50暫時退榜，取而代之入榜的台股ETF僅有元大MSCI台灣（006203）、永豐台灣加權（006204），名列第3、5名。此外，在A股

▲明年報綜所稅有3利多，相關民眾可以檢附證明運用，達到減稅的小確幸。（圖／記者湯興漢攝）
記者吳靜君／台北報導
政府提高4大扣除額，包含標準扣除額、薪資扣除額、身心障礙扣除額以及幼兒學前扣除額，民眾今年報稅時很有感之外。迎接新的一年到來，政府再增加3大減稅利多，民眾可以準備相關的檢附文件、明（2020）年報稅時檢視是否可以運用，讓自己的小確幸溫暖再升級。
一，林若亞條款上路　薪資扣除額可以核實減稅
今年7月已經公布修正所得稅法，未來薪資扣除額可以二擇一，一為定額減除20萬元；或者是可以選擇核實扣除（又稱為林若亞條款），其適用範圍及認定辦法如下包含職業專用的服裝費，包含購置、租用及清潔費等；職業所需的進修費用；以及職業所需的工具，其效能非2年內所能耗竭且支出超過8萬元者，前述三項可以扣除的各為所得的3%為限。
北區國稅局舉例，以某甲的薪資收入900萬元，今年若自費購買設計軟體90萬元，甲君可列報職業上工具支出上限為27萬元（900萬元×3%）低於實際支出按平均法分3年免列殘值可攤提折舊30萬元，但高於當年度薪資所得特別扣除額20萬元，甲君可檢附相關證明文件自其薪資收入900萬元中核實減除必要費用27萬元，多了7萬元的可扣除額，申報薪資所得就變成了873萬元。
二，長照扣除額　每人每年12萬元
納稅義務人、配偶或者是受扶養的親屬，若有符合衛福部所公告須長期照顧的身心失能者，明年申報所得稅時，每人多12萬元可列報長照扣除額。
但是民眾應該檢附的證明文件包含，如果有聘僱外籍看護工，就可以檢附聘僱許可函；如果在家自行照顧者，可以檢附病症失能診斷證明暨巴氏量表影本或者是若符合特定身心障礙重度等級項目或鑑定向度之一，就可附身心障礙證明、手冊等。
▲申請長照扣除額應檢附的文件。（圖／財政部提供）
如果符合長服法失能等級為第2級至第8級且使用長期照顧給付及支付基準服務者，只要檢附今年度使用服務的繳費收據影本、註記特約服務單位名稱、失能者姓名、身分證字號及失能等級等資料。
入住住宿式服務機構全年達90日者，檢附入住累計達90日的繳費收據影本，並註記機構名稱、住民姓名、身分證字號、入住期間及床位類型等資料。
國稅局洽相關單位　已歸戶者可以不用再行檢附
為了提升便民服務，國稅局已洽衛生福利部、勞動部及國軍退除役官兵輔導委員會等相關部會蒐集長照扣除額資料，納稅義務人在明（2020）年5月報綜

▲東森社交電商高峰會 王令麟霸氣喊「成為台灣直銷第一名」。(採訪撰稿／記者黃稜涵；攝影剪輯／記者謝盛帆)
▲2019年截至目前單就網連通的營業金額就高達近5億，分潤金額已超過2000萬，業績超過百萬以上的夥伴超過29位，金額最高的夥伴銷售額更達850萬，讓東森全球新連鎖執行副總顧武男開心地表示，台灣社交電商第一名就是東森！（圖／東森全球新連鎖事業提供，以下同）
記者萬玟伶／台中報導
東森全球新連鎖事業12月28日（六）於台中金典酒店舉辦今年第三次「東森社交電商大會」，現場聚集千名會員與經銷夥伴熱情參與。年終歲末時刻，大會也公布2019年創造的營運佳績，東森全球新連鎖執行副總顧武男表示，「今年是快速成長的一年！截至目前網連通的營業金額達近5億元，用戶人數也突破14萬，整體產值與去年相比，營業額與會員數皆達15倍的成長。以社交電商這個產業來看，其餘公司的營業數字距離我們都還有一段距離，可以說台灣社交電商第一名就是東森！」
大會現場，顧武男也與夥伴們一同回顧今年東森全球新連鎖事業針對「社交電商」開發的全新功能與商業模式。今年3月第一場社交電商大會，東購App新增「我要賣」功能，營業額就從這裡開始成長，並創造了3月份共5600萬元的營業額，當時產生的分潤將近1000萬元。今年7月整合了更多關係企業，包括台灣票券館、東森保代、券集商城等資源，給用戶夥伴多方位經營。此後，網連通也推出「美麗站長」新商業模式，並引進自然美新開發的保養品牌Angelita，做為廣大店主們的第二收入來源，為親友打造美麗的同時，最高還能得到30%的誘人分潤！當時會員人數來到10萬，累積成交總額突破2.5億元、產生的分潤也累積千萬元以上。
▲東森全球新連鎖事業於12月28日（六）在台中金典酒店舉辦今年第三次社交電商大會，聚集網連通會員熱情參與。
回顧今年，東森全球新連鎖事業以創新概念推出「美麗站長」新商模。然而，東森相關企業眾多，有集結台灣優質廠商品牌的東森電視購物、購物網，還有保養美妝的自然美，網連通今年再與東森保代合作，那麼還有什麼是電商市場沒有的，顧武男開心地表示，「答案就是房屋！」
顧武男認為，房屋產業目前仍偏傳統保守，過去幾十年都是透過房仲帶客戶看房子賣房子，中間有很多複雜流程。他想，既然社交電商愈漸成熟，可分享有形產品、無形服務，那麼更可以把觸角延伸到其他產業，東森有自己的房仲產業「東森房屋」

▲受尾牙旺季影響，住宿餐飲業求才數激增1萬5,380人。（圖／記者張一中攝）
記者余弦妙／台北報導
行政院主計總處日前公布國內108年11月失業率為3.73％，較108年10月下降0.04個百分點，較前一年同月上升0.03個百分點。根據勞動部勞動力發展署「台灣就業通」就業服務資料庫統計，「住宿及餐飲業」受到年底尾牙、聖誕及跨年餐飲旺季的加持，11月新登記求才人數達15,380人，較10月成長9.38％，成長人數居各行業之冠。
台灣就業通指出，108年11月各公立就業服務機構新登記求職人數總計54,099人，較10月減少8.31％，新登記求才人數為96,318人，較10月減少2.54％，平均每一位求職者有1.78個工作機會(即求供倍數為1.78倍)。
此外，台灣就業通也發現，隨著跨年、春節即將到來，與節慶商機相關的產業均逆勢成長。「批發及零售業」受惠於耶誕節及明年農曆春節鋪貨旺季來臨，帶動相關就業需求，11月新登記求才人數為13,402人，創下近3年同月新高紀錄，月增幅與年增幅分別達2.35％與8.03％。
在「批發及零售業」方面，近4成職缺集中在第一線的「商店銷售有關人員」，目前美廉社、家樂福、全聯實業、惠康百貨、愛買等量販超市均有大規模徵才。因應春節旺季，家樂福提早釋出上百位寒假工讀生職缺，工作性質涵蓋收銀人員、營運服務、購物車管理、網購營運等不同類型。
另外，在「住宿及餐飲業」方面，近5成職缺集中在第一線的「其他餐飲服務人員」，為搶攻耶誕、跨年商機，目前海底撈火鍋、雲雀國際、鑫鼎王、王座國際餐飲、豆府、饗賓餐旅、肉多多等餐飲集團均徵才上百人，其中饗賓餐旅的開飯川食堂招募的外場店主任薪資上看45K；而雲雀國際提供員工團保、跨域津貼、閉店津貼、眷屬津貼與各式獎金等福利。
而在農曆年前也是春節大掃除的旺季，「支援服務業」的人力需求亦較10月增加10.53％，包括媽咪樂居家服務集團、晶緻家、新高清潔服務、金采清潔、樂家庭、潔凌企業等均急徵相關清掃人員，其中潔凌企業徵求的長期鐘點管家，薪資上看48K。
----------------------------------------------------------------------

[68] 影／運用大數據賣出上億片面膜　軒郁攜手雲端科技大廠加速驗證

▲軒郁國際總經理楊尚軒。（圖／記者姚惠茹攝） 
記者姚惠茹

▲勞動部。（圖／記者余弦妙攝）
記者余弦妙／台北報導
勞動部今（27）日表示，為保障具投票權的勞工得以行使參政權，勞動部已指定選舉投票日為「勞動基準法」所定放假日，因此，109年1月11日（星期六）2020總統大選投票當日，具有投票權且原屬工作日的勞工，雇主應依法讓勞工放假一日，工資照給。至於具投票權但當日原屬勞工的休息日或例假者，因勞工本得於當日行使投票權，因此不另外給假。
勞動部表示，放假「一日」指的是午前零時至午後12時連續24小時。由於投票權僅得於投票當日行使，性質與一般國定假日不同，因此，投票日不得再與其他工作日對調實施。
勞動部進一步說明，雇主如果徵得具投票權勞工同意於投票日出勤工作，應不妨礙其投票，並應加給當日出勤工資，至於計算方式分為下列兩種類型：
一、 原屬勞工的工作日：應加倍發給出勤時段的工資。舉例而言：勞雇雙方約定當天出勤8小時，勞工外出投票2小時，返回工作6小時，雇主須加給6小時的工資。
二、 原屬勞工的休息日：視工作時數依「勞動基準法」第24條第2項有關休息日出勤加給之規定計算。
勞動部條件司司長謝倩蒨舉例，一名勞工月薪3萬6000元，換算日薪為1200元、時薪150元，如在周六為工作日，投票日當天出勤6小時上班，則當日加班費應給予900元；若當日是休息日，雇主經徵得勞工同意後出勤6小時，加班費應給予1400元，至於部分工時工作者，則應給予2倍的時薪。
最後，勞動部再次提醒，雇主未依法給假或給薪者，可處新臺幣2萬元以上，最高100萬元罰鍰，並應補給工資。勞工權益如有受損害者，可就近向工作所在地勞工行政主管機關（即當地直轄市、縣市政府之勞工或社會局、處）申訴，以維權益。
----------------------------------------------------------------------

[79] 有線電視授權費爭議大和解！練台生、全國敲定新系統MG10只收2年

----------------------------------------------------------------------

[80] 遠見／高希均：發揚君子精神，人生典範在分享

文╱高宜凡　攝影／張智傑
「人生的樂趣在學習，人生的收穫在奉獻，人生的意義在創造。」遠見．天下文化事業群創辦人暨董事長高希均代表主席團在開場致詞時，以今年獲頒「

▲ 中央銀行。（圖／記者李毓康攝）
記者李瑞瑾／台北報導
央行今（27）日公布本國銀行9月國家風險統計，9月底本國銀行外國債權直接風險餘額為4,342億美元，較108年6月底減少11億美元或0.26%，主要是對銀行部門債權減少所致；由於部分直接債務人與其最終債務人或保證人分屬不同國家，經風險移轉後，9月底最終風險淨額為4,172億美元，較108年6月底增加24億美元或0.57%。
外國債權直接風險依部門別分析，以非銀行之私人部門占62.4%為主，銀行占29.79%次之，公共部門則占7.8%；依國家類別分析，以已開發國家占52.5%為主，開發中國家及境外中心分別占24.67%及22.28%；依地區別分析，以亞洲及太平洋地區占47.5%為主，其次為美洲及加勒比海地區占28.86%，歐洲地區占20.42%。
另外，本國銀行6月底外國債權直接風險前十大國家（地區），依序為美國、中國大陸、盧森堡、香港、日本、澳大利亞、英國、開曼群島、新加坡及英屬西印度群島，合計3,197億美元，占外國債權直接風險餘額之73.64%。
其中，9月底對中國大陸直接風險為438億美元，較6月底減少14億美元（或3.01%）；最終風險為651億美元，較6月底減少28億美元(或4.13%)
央行官員表示，美國債權直接風險因資金拆存及證券化商品投資增加，續創歷史新高紀錄，且已連續17季位居外國債權直接風險第一大國；而對中國大陸直接風險減少，除了受金融債券投資跟資金拆存減少這兩項因素之外，也因人民幣貶值，致使換算成美元統計之後的債權金額減少。
----------------------------------------------------------------------

[88] 爭取5G入場不手軟！競標14天達746.42億　30日挑戰800億

----------------------------------------------------------------------

[89] 歷經3屆奧運挫敗...國巨陳泰銘二公主陳少曼取得2020東奧馬術門票　

▲國巨二公主陳少曼取得東奧馬術門票。（圖／陳少曼提供）
記者周康玉／台北報導
國巨（2327）董事長陳泰銘次女陳少曼於日前（12/21）在沙烏地阿拉伯舉辦的奧運個人積分賽中，已取得絕對領先，陳少曼今（27）日對外表示，已經

In [8]:
# 關閉瀏覽器
browser.close()